In [0]:
print("Imports")
import numpy as np
import pandas as pd
# import sklearn as skl
from sklearn import linear_model
from sklearn import neural_network as nnet
from sklearn import svm
from sklearn import ensemble

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

from math import sqrt

# !pip install category_encoders
# import category_encoders as ce

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D as a3d
from google.colab import files
from datetime import datetime as dt
print(dt.now())

In [0]:
print ("Data loads")
training_path = "https://raw.githubusercontent.com/johnsl01/income/master/incomeknown.csv"
train_df = pd.read_csv(training_path)
test_path = "https://raw.githubusercontent.com/johnsl01/income/master/incomeunknown.csv"
test_df = pd.read_csv(test_path)
print(dt.now())

In [0]:
print("Rename columns to not contain spaces")

newnames = {"Year of Record" : "Year",
           "Size of City" : "CitySize",
           "University Degree" : "Degree",
           "Wears Glasses" : "Glasses",
           "Hair Color" : "Hair",
           "Body Height [cm]" : "Height",
           "Income in EUR" : "Income"
          }

train_df.rename(columns=newnames, inplace=True)
test_df.rename(columns=newnames, inplace=True)
print(dt.now())

In [0]:
print("hot-one encoding gender")
train_df.Gender.fillna("u", inplace=True) # fillna() replaces Null values
test_df.Gender.fillna("u", inplace=True)

cols = ["GenderFemale", "GenderMale", "GenderOther", "GenderUnknown", "Gender0", "GenderNull"]
genders = ["female", "male", "other", "unknown", "0", "u"]

for i in range(len(cols)) : 
  train_df[cols[i]] = train_df["Instance"]
  train_df.loc[train_df.Gender == genders[i], cols[i]] = 1
  train_df.loc[train_df.Gender != genders[i], cols[i]] = 0
  
  test_df[cols[i]] = test_df["Instance"]
  test_df.loc[test_df.Gender == genders[i], cols[i]] = 1
  test_df.loc[test_df.Gender != genders[i], cols[i]] = 0
#end for

#print(test_df["GenderNull"].describe())
print(dt.now())

In [0]:
print("hot-one encoding hair colour")
train_df.Hair.fillna("U", inplace=True)
test_df.Hair.fillna("U", inplace=True)

cols = ["HairBlack", "HairBrown", "HairRed", "HairBlonde", "HairUnknown", "Hair0", "HairNull"]
colours = ["Black", "Brown", "Red", "Blonde", "Unknown", "0", "U"]

for i in range(len(cols)) :
  train_df[cols[i]] = train_df["Instance"]
  train_df.loc[train_df.Hair == colours[i], cols[i]] = 1
  train_df.loc[train_df.Hair != colours[i], cols[i]] = 0
  
  test_df[cols[i]] = test_df["Instance"]
  test_df.loc[test_df.Hair == colours[i], cols[i]] = 1
  test_df.loc[test_df.Hair != colours[i], cols[i]] = 0
#end for

#print(train_df["HairBlack"].describe())
print(dt.now())

In [0]:
print("renaming Profession values")
train_df.Profession.fillna("Unknown", inplace=True)

test_df.Profession.fillna("Unknown", inplace=True)
print(dt.now())

In [0]:
print("categorically encoding degree values")

train_df.Degree.replace("PhD", 5, inplace=True)
train_df.Degree.replace("Master", 4, inplace=True)
train_df.Degree.replace("Bachelor", 3, inplace=True)
train_df.Degree.replace("No", 2, inplace=True)
train_df.Degree.replace("0", 6, inplace=True)
train_df.Degree.fillna(1, inplace=True)

test_df.Degree.replace("PhD", 5, inplace=True)
test_df.Degree.replace("Master", 4, inplace=True)
test_df.Degree.replace("Bachelor", 3, inplace=True)
test_df.Degree.replace("No", 2, inplace=True)
test_df.Degree.replace("0", 6, inplace=True)
test_df.Degree.fillna(1, inplace=True)

# new Degree values
# 4 (was PhD), 3 (was Master), 2 (was Bachelor), 1 (was 0,Null), 0 (was No)

print(dt.now())

print ("Validate this - is there a correlation and is it linear - one-hot may be better")
print ("  or consider encoding the nulls as a separate feature 0/1")

In [0]:
for degre in [1, 2, 3, 4, 5, 6] :
  
  print (degre, round(train_df.loc[(train_df.Degree == degre)].Income.median(),0), round(train_df.loc[(train_df.Degree == degre)].Income.mean(),0), round(train_df.loc[(train_df.Degree == degre)].Income.std(),0) )
  
print(dt.now())


In [0]:
'''
for degre in [0, 1, 2, 3, 4, 5] :
  
  print (degre, train_df.loc[(train_df.Degree == degre)].Income.median(), train_df.loc[(train_df.Degree == degre)].Income.median())
  
print(dt.now())
'''

In [0]:
print("replacing Null ages and years with median")
# median age
train_med_age = train_df["Age"].median()
print("train median age = ", train_med_age)
test_med_age = test_df["Age"].median()
print("test median age = ", test_med_age)
mean_of_age_medians = (train_med_age + test_med_age)/2
print("mean = ", mean_of_age_medians)

print("replacing age...")
train_df.Age.fillna(mean_of_age_medians, inplace=True)
test_df.Age.fillna(mean_of_age_medians, inplace=True)

# median year
train_med_yr = train_df["Year"].median()
print("train median year = ", train_med_yr)
test_med_yr = test_df["Year"].median()
print("test median year = ", test_med_yr)
mean_of_yr_medians = (train_med_yr + test_med_yr)/2
print("mean = ", mean_of_yr_medians)

print("replacing year...")
train_df.Year.fillna(mean_of_yr_medians, inplace=True)
test_df.Year.fillna(mean_of_yr_medians, inplace=True)

print(dt.now())

print ("Again - validate these two - may be a better choice - MICE etc.")
print ("   Is there a correlation - explore this more")

In [0]:
'''
print("Capping city size at 2mil")
# only 19 people live in cities above 2mil in train_df

twomil = 2000000
train_df["CitySize"].values[train_df["CitySize"] > twomil] = twomil
test_df["CitySize"].values[test_df["CitySize"] > twomil] = twomil

print(dt.now())

print ("Is there any relevant correlation here - capping result may makes sense")
print ("but is there any point in capping an independent variable ?")
'''

In [0]:
print("Adding column that labels instances where citysize<=100k")

hunk = 100000
train_df["LivesInTown"] = train_df["Instance"]
train_df["LivesInTown"].values[train_df["CitySize"] <= hunk] = 1
train_df["LivesInTown"].values[train_df["CitySize"] > hunk] = 0


test_df["LivesInTown"] = test_df["Instance"]
test_df["LivesInTown"].values[test_df["CitySize"] <= hunk] = 1
test_df["LivesInTown"].values[test_df["CitySize"] > hunk] = 0

print(dt.now())

print("There is an identified corelation here")

In [0]:
print("encoding country data")

overall_md = train_df["Income"].median()
overall_mean = train_df["Income"].mean()

# if not particularly acurate, try using country_count too,
# since smaller sample sizes seem to have much higher wages

# add new column that holds the difference between
# the mean income for a country and the mean overall
# countries in Test that are not in Train will be left as 0
# (ie. mean overall for train)
train_df["CountryMed"] = train_df["Instance"]
train_df = train_df.assign(CountryMed = 0)
train_df["CountryMean"] = train_df["Instance"]
train_df = train_df.assign(CountryMean = 0)

test_df["CountryMed"] = train_df["Instance"]
test_df = test_df. assign(CountryMed = 0)
test_df["CountryMean"] = train_df["Instance"]
test_df = test_df. assign(CountryMean = 0)

for cntry in train_df.Country.unique():
  # count = len(train_df.loc[(train_df.Country == cntry)])
  country_md = train_df.loc[(train_df.Country == cntry)].Income.median()
  country_mean = train_df.loc[(train_df.Country == cntry)].Income.mean()
  median_diff = country_md - overall_md
  mean_diff = country_mean - overall_mean
  # replacing CountryMean (was 0)
  train_df.loc[(train_df.Country == cntry), "CountryMed"] = median_diff
  train_df.loc[(train_df.Country == cntry), "CountryMean"] = mean_diff
  test_df.loc[(test_df.Country == cntry), "CountryMed"] = median_diff
  test_df.loc[(test_df.Country == cntry), "CountryMean"] = mean_diff
  # end for
#end for
print(dt.now())

# there is an identified corelation between countries with fewer sample and 
# income.

# perhaps add a sample count col - and this could be the sample count from the 
# whole data set as this is likely the way it was introduced - but small sample
# sizes may be therefore inaccurate in test if under or over 
# represeneted and result in over fitting. 

In [0]:
print("encoding profession data")

overall_md = train_df["Income"].median()
overall_mean = train_df["Income"].mean()

# if not particularly acurate, try using prof_count too?

# add new column that holds the difference between
# the mean income for a country and the mean overall
# countries in Test that are not in Train will be left as 0
# (ie. mean overall for train)
train_df["ProfMed"] = train_df["Instance"]
train_df = train_df.assign(ProfMed = 0)
train_df["ProfMean"] = train_df["Instance"]
train_df = train_df.assign(ProfMean = 0)

test_df["ProfMed"] = train_df["Instance"]
test_df = test_df. assign(ProfMed = 0)
test_df["ProfMean"] = train_df["Instance"]
test_df = test_df. assign(ProfMean = 0)

for prof in train_df.Profession.unique():
  # count = len(train_df.loc[(train_df.Profession == prof)])
  prof_md = train_df.loc[(train_df.Profession == prof)].Income.median()
  prof_mean = train_df.loc[(train_df.Profession == prof)].Income.mean()
  mean_diff = prof_mean - overall_mean
  median_diff = prof_md - overall_md
  # replacing CountryMean (was 0)
  train_df.loc[(train_df.Profession == prof), "ProfMed"] = median_diff
  train_df.loc[(train_df.Profession == prof), "ProfMean"] = mean_diff
  test_df.loc[(test_df.Profession == prof), "ProfMed"] = median_diff
  test_df.loc[(test_df.Profession == prof), "ProfMean"] = mean_diff
  # end for
#end for
print(dt.now())

# need to check this correlation - ther3e may be a more obvious link
# the high counts in the 'p', 'r', 's' etc professions 
# and low counts in 'a', 'b'. 'c' suggest artifical data.

In [0]:
print("normalise data")
# merge tables in order to normalise properly
(split_point, _) = train_df.shape
all_df = pd.concat([train_df, test_df], ignore_index=True)
#print(all_df.describe())

# selects columns to normalise and normalises
all_num = all_df[
    ["Year", "Age", "CountryMed", 
     "CountryMean", "ProfMed", "ProfMean", 
     "CitySize", "Degree", "Height"]]
all_norm = (all_num - all_num.min())/ (all_num.max()-all_num.min())

#print(all_norm.describe())

# (rows, _) = all_norm.shape
# tmp = np.ones(rows)
# all_norm.insert(0, "Ones", tmp)

# add all data that's already binary data
all_norm["Glasses"] = all_df["Glasses"]
all_norm["LivesInTown"] = all_df["LivesInTown"]
all_norm["GenderMale"] = all_df["GenderMale"]
all_norm["GenderFemale"] = all_df["GenderFemale"]
all_norm["GenderUnknown"] = all_df["GenderUnknown"]
all_norm["Gender0"] = all_df["Gender0"]
all_norm["GenderNull"] = all_df["GenderNull"]
all_norm["HairBlack"] = all_df["HairBlack"]
all_norm["HairBrown"] = all_df["HairBrown"]
all_norm["HairBlonde"] = all_df["HairBlonde"]
all_norm["HairRed"] = all_df["HairRed"]
all_norm["HairUnknown"] = all_df["HairUnknown"]
all_norm["Hair0"] = all_df["Hair0"]
all_norm["HairNull"] = all_df["HairNull"]

# split data
train_norm = all_norm.head(split_point)
test_norm = all_norm.tail(len(all_norm) - split_point)
#print(train_norm.describe())
#print(test_norm.describe())
print(dt.now())

In [0]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model

# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

#   25    : 0.2497450605388929 - 0.25324189201654834 -       11 secs
#   50    : 0.6616425189653192 - 0.73051652138610000
#  100    : 0.7827040270315250 - 0.78551363329530390 -     1:04
#  200    : 0.8195381668117979 -                           4:04

# learning from previous - makes timing difficult - and isn't that effective
# and early stopping excludes data so this is not helpful in this case
# warm_start=False, early_stopping=False,
#   25    : 0.28669907374694426 - 0.3143078977278916 -       11
#   50    : 0.73137165225086110 - 0.7229390424910851 -       31          36
#  100    : 0.78481393684434940 - 0.7857755827423643 -     1:06        1:11
#  200    : 0.7903792664668625 -                           4:27
#  500    : 0.8450908332162614 -                          14:06

# default layers make no sense try (20,10)
# hidden_layer_sizes=(20,10),
# first hidden layer matched to inputs - and use a second as the data has arbitrary
# thresholds
# (number of features is a good number of neurons per layer - deeper gives more sophistication)
#    25   : 0.7887734639019761 - 0.7809109537904840 -         06          06

#   100   : 0.7947890140674736 - 0.7922373078596026 -       0:46        0:33

#   500   : 0.8675915246886190 -                            4:44

#  2500   : 0.7923683126864547 - 0.8622292552558385         0:39 ?      8:34
 
# hidden_layer_sizes=(20,10,4),
# early_stopping=True,  

#    25   : 0.8329299620790784 - 0.7815451498511101         0:07        0:07
 
#   100   : 

# try verbose=True on a small example


# hidden_layer_sizes=(20,10,5),
# learning_rate_init=0.001,
# early_stopping=True,

#    25   : 0.19057347109521827  0:07

#   100   : 0.78176263624963760  0:33  

#   500   : 0.85071602031931740  5:39

#  2500   : 0.85154263342489110  2:29 276


# hidden_layer_sizes=(20,20),

#  2500

In [0]:
'''
# add some more loops 
# and complete investigation of data = ref above


# for hidden in [(5,), (10,), (20,), (30,), (10,5), (10,10), (15,10), (20, 10), (20,20), (15,10,5), (20,10,5), (20,20,10), (20,20,20), (30,20,20), (30,20,10,5), (40,30,20,10)] :
for dpth in [2,3,4,5,6,7,8,9,10] :
  for iters in [200, 500, 1000] :
    print ("\n    ",dpth,iters)
    # print ("\n    ",hidden,iters)
    print("      Fitting model and checking score on Known data" , iters, "iterations")
    Known = train_norm.copy()
    y = train_df["Income"].copy()


    #mod = linear_model.BayesianRidge(copy_X=False, tol=0.00001, n_iter=1000)
    # 0.7349053641228902
        
    # mod = nnet.MLPRegressor(
    #     hidden_layer_sizes= hidden,
    #     tol=0.00001, n_iter_no_change=25,
    #     warm_start=False, early_stopping=True,
    #     learning_rate="adaptive", learning_rate_init=0.001,
    #     max_iter=iters)
        
    # mod = ensemble.AdaBoostRegressor( n_estimators=100 )
    # doesn't work   
    
    # mod = svm.LinearSVR( tol=1e-5, max_iter =10000)
    # doesn't work
    
    # mod = ensemble.RandomForestRegressor(max_depth=7, n_estimators=1000)
    # 0.8288730954198963 in 6:06 - (max_depth=7, n_estimators=1000)
    mod = ensemble.RandomForestRegressor(max_depth=dpth, n_estimators=(iters*2))
    
    
    dtstart = dt.now()
    print("      model created ", dt.now())
    mod.fit(Known, y)
    print("      model   fit   ", dt.now())

    print("      ", mod.score(Known, y))

    print ("      ", dt.now() - dtstart)
    # print ("      ", mod.n_iter_)

    # print("      ", dt.now())
  
  # end for
  print('end   ', hidden, dt.now())
# end for
print('end', iters, dt.now())
'''

In [0]:
'''
# new section to use test/train split validation  
# to find the model complexity at which overfitting begins to occur.

# We also need to understand how this overfitting point moves with dataset size
# as the full model (i.e trained on all the data) which we eventually produce
# may need to be slightly more complex,
# so we look for the overfitting point on two different sizes of test data.

# the models we are working with have multiple parameters which govern their 
# behaviour - so there is a gradual approach towards the overfitting point 

# in general with the number of samples several orders of magnitiude larger
# than the number of features we have to push the models pretty hard to get to 
# an overfit

# initialise results vectors

train1R2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
train2R2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
train3R2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
test1R2 =  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
test2R2 =  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

# pull in our full Known data (already engineered and normalised)
# and our full known result

# df = train_norm.copy()
# y = train_df["Income"].copy()

# X_train1, X_test1, y_train1, y_test1 = train_test_split(df, y, test_size=0.5)

# X_train2, X_test2, y_train2, y_test2 = train_test_split(df, y, test_size=0.2)

# run 10 samples ( modify parameters to suit needs)
# for i in [8,9] :
for i in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19] :
  
  df = train_norm.copy()
  y = train_df["Income"].copy()

  X_train1, X_test1, y_train1, y_test1 = train_test_split(df, y, test_size=0.5)

  X_train2, X_test2, y_train2, y_test2 = train_test_split(df, y, test_size=0.2)
  
  X_train3 = df
  y_train3 = y
  
  # Random Forrest : 
  dpth = [2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,11,11]
  tres = [500, 1000, 500, 1000, 500, 1000, 500, 1000, 500, 1000, 500, 1000, 500, 1000, 500, 1000, 500, 1000, 500, 1000]
  print(i,"   model fit   ", dept[i], ' deep ', tres[i], ' trees, ', dt.now())
  
  dtstart = dt.now()
  mod1 = ensemble.RandomForestRegressor(max_depth=dpth[i], n_estimators=tres[i])
  mod1.fit(X_train1, y_train1)
  train1R2[i] = mod1.score(X_train1, y_train1)
  test1R2[i] = mod1.score(X_test1, y_test1)
  print ('     Train-1 ', train1R2[i], '    Test-1 ',test1R2[i]  )
  
  print("     ", dt.now() - dtstart)
  dtstart = dt.now()
  
  mod2 = ensemble.RandomForestRegressor(max_depth=dpth[i], n_estimators=tres[i])
  mod2.fit(X_train2, y_train2)
  train2R2[i] = mod2.score(X_train2, y_train2)
  test2R2[i] = mod2.score(X_test2, y_test2)
  print ('     Train-2 ', train2R2[i], '    Test-2 ',test2R2[i]  )
  
  print("     ", dt.now() - dtstart)
  dtstart = dt.now()
  
  mod3 = ensemble.RandomForestRegressor(max_depth=dpth[i], n_estimators=tres[i])
  mod3.fit(X_train3, y_train3)
  train3R2[i] = mod3.score(X_train3, y_train3)
  # test2R2[i] = mod2.score(X_test2, y_test2)
  print ('     Train-3 ', train3R2[i]  )
  
  print("     ", dt.now() - dtstart)
  
print ( " END ")  
  
'''

In [0]:
# generalised test model complexity section

# model type neural network so
# instance is ( max_iterations:int, hidden_layer_config:tupple) : tupple
'''
testrange=[
             (300,  (10,)            )
           , (300,  (15,)            )
           , (500,  (15,)            )
           , (500,  (20,)            )
           , (500,  (30,)            )
           , (500,  (10,5)           )
           , (500,  (10,10)          )
           , (500,  (15,10)          )
           , (1000, (15,10)          )
           , (1000, (20,10)          )
           , (1000, (20,20)          )
           , (1000, (15,10,5)        )
           , (1000, (20,10,5)        )
           , (1000, (20,20,10)       )
           , (1000, (20,20,20)       )
           , (2000, (20,20,20)       )
           , (2000, (30,20,20)       )
           , (2000, (30,20,10,5)     )
           , (2000, (40,30,20,10)    )
           , (2000, (40,30,20,10,5)  )
           , (5000, (40,30,20,10,5)  )
           , (5000, (50,40,30,20,10) )
           , (5000, (50,50,40,40,30) )
           , (5000, (50,50,50,50,50) )
            ]
'''
testrange=[
             (750,  (50,50,50,50,50)       )
           , (600,  (60,60,60,60,60,60)    )
           , (500,  (70,70,70,70,70,70,70) )
          ]

testlen = len(testrange)
print (testlen, 'test cases')  

train1R2 = np.zeros(testlen)
train2R2 = np.zeros(testlen)
train3R2 = np.zeros(testlen)
test1R2 =  np.zeros(testlen)
test2R2 =  np.zeros(testlen)
    
for i in range(testlen) :
  
  # pick up my model data
  model = testrange[i]
  iters = model[0]
  hidden = model[1]
  
  print (i, "Iterations : ", iters, " hidden layer config : ", hidden )
  
  # pick up my sample data
  # resetting data for each model
  df = train_norm.copy()
  y = train_df["Income"].copy()

  X_train1, X_test1, y_train1, y_test1 = train_test_split(df, y, test_size=0.5)

  X_train2, X_test2, y_train2, y_test2 = train_test_split(df, y, test_size=0.2)
  
  X_train3 = df
  y_train3 = y
  
  # need three identical models
  mod1 = nnet.MLPRegressor(
                            hidden_layer_sizes= hidden
                          , max_iter=iters
                          , tol=0.00001
                          , n_iter_no_change=25
                          , warm_start=False
                          , early_stopping=True
                          , learning_rate="adaptive"
                          , learning_rate_init=0.0001
                          )  

  mod2 = nnet.MLPRegressor(
                            hidden_layer_sizes= hidden
                          , max_iter=iters
                          , tol=0.00001
                          , n_iter_no_change=25
                          , warm_start=False
                          , early_stopping=True
                          , learning_rate="adaptive"
                          , learning_rate_init=0.0001
                          )
  mod = nnet.MLPRegressor(
                            hidden_layer_sizes= hidden
                          , max_iter=iters
                          , tol=0.00001
                          , n_iter_no_change=25
                          , warm_start=False
                          , early_stopping=True
                          , learning_rate="adaptive"
                          , learning_rate_init=0.0001
                          )
  
  dtstart = dt.now()
  
  mod1.fit(X_train1, y_train1)
  train1R2[i] = mod1.score(X_train1, y_train1)
  test1R2[i]  = mod1.score(X_test1, y_test1)
  train1RMS = sqrt ( mean_squared_error(y_train1, mod1.predict(X_train1)))
  test1RMS  = sqrt ( mean_squared_error(y_test1, mod1.predict(X_test1)))
  print ('      Train-1 ', train1R2[i], '     Test-1 ',test1R2[i]  )
  print (' RMSE-Train-1 ', train1RMS,   'RMSE-Test-1 ',test1RMS )
  print (mod1.n_iter_)
  print("     ", dt.now() - dtstart)
  
  dtstart = dt.now()
  
  mod2.fit(X_train2, y_train2)
  train2R2[i] = mod2.score(X_train2, y_train2)
  test2R2[i]  = mod2.score(X_test2, y_test2)
  train2RMS = sqrt ( mean_squared_error(y_train2, mod2.predict(X_train2)))
  test2RMS  =  sqrt ( mean_squared_error(y_test2, mod2.predict(X_test2)))
  print ('      Train-2 ', train2R2[i], '     Test-2 ',test2R2[i]  )
  print (' RMSE-Train-2 ', train2RMS,   'RMSE-Test-2 ',test2RMS )
  print (mod2.n_iter_)
  print("     ", dt.now() - dtstart)
  dtstart = dt.now()
  
  mod.fit(X_train3, y_train3)
  train3R2[i] = mod.score(X_train3, y_train3)
  train3RMS = sqrt ( mean_squared_error(y_train3, mod.predict(X_train3)))
  # test3R2[i] = mod.score(X_test2, y_test2)
  print ('      Train-3 ', train3R2[i]  )
  print (' RMSE-Train-3 ', train3RMS )
  print (mod.n_iter_)
  print("     ", dt.now() - dtstart)
  
  
print ( " END ")
  
  
  

In [0]:
# plot the fitting data

tr1 = np.zeros(testlen)
tr2 = np.zeros(testlen)
tr3 = np.zeros(testlen)
ts1 = np.zeros(testlen)
ts2 = np.zeros(testlen)

x = range(testlen)

for i in x : 
  tr1[i] = 1-train1R2[i]
  tr2[i] = 1-train2R2[i]
  tr3[i] = 1-train3R2[i] 
  ts1[i] = 1-test1R2[i]
  ts2[i] = 1-test2R2[i]
  
plt.figure(figsize=(16,8), dpi=80)  
plt.plot (x, tr1, color='cyan',  label="train 1 -  50%")
plt.plot (x, tr2, color='pink',  label="train 2 -  80%")
plt.plot (x, tr3, color='green', label="train 3 - 100%")
plt.plot (x, ts1, color='blue',  label="test 1 - 50%")
plt.plot (x, ts2, color='red',   label="test 2 - 80%")
plt.legend();
plt.title("Looking for overfitting point")
plt.xlabel("model complexity \n adding iterations and hidden layers")
plt.ylabel("1-R2")

# plt.savefig('nn-fit-a.pdf')

# files.download('nn-fit-a.pdf')


In [0]:
'''
print (mod.n_iter_)
# Known.describe()
'''

In [0]:

print("Predicting unknown data")
Unknown = test_norm.copy()
u_income = mod.predict(Unknown)
u_mean = u_income.mean()
u_std = u_income.std()
k_mean = train_df.Income.mean()
k_std = train_df.Income.std()
print("MEAN: k=", k_mean, "\n      u=",u_mean)
print("STDD: k=", k_std, "\n      u=", u_std)
print(dt.now())


In [0]:
'''
submission = pd.DataFrame(columns=["Instance", "Income"])
submission["Instance"] = test_df["Instance"].copy()
submission["Income"] = u_income.copy()
submission.to_csv(r"submission.csv",index=None, header=True)
files.download("submission.csv")
'''

In [0]:
"""
# looking at mean/median for different countries
c = "Solomon Islands"
ggg = train_df.loc[(train_df.Country == c)].Income.mean()
print(ggg)
hhh = train_df.loc[(train_df.Country == c)].Income.median()
print(hhh)
print(train_df.Income.mean())
print(train_df.Income.median())
print(train_df.loc[(train_df.Country == c)].Income)
"""

#### https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model

#### https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

   25    : 0.2497450605388929 - 0.25324189201654834 -       11 secs

   50    : 0.6616425189653192 - 0.73051652138610000

  100    : 0.7827040270315250 - 0.78551363329530390 -     1:04

  200    : 0.8195381668117979 -                           4:04

#### learning from previous - makes timing difficult - and isn't that effective
#### and early stopping excludes data so this is not helpful in this case
#### warm_start=False, early_stopping=False,

25    : 0.28669907374694426 - 0.3143078977278916 -       11

50    : 0.73137165225086110 - 0.7229390424910851 -       31          36

100    : 0.78481393684434940 - 0.7857755827423643 -     1:06        1:11

200    : 0.7903792664668625 -                           4:27

500    : 0.8450908332162614 -                          14:06

#### default layers make no sense try (20,10)
#### hidden_layer_sizes=(20,10),
#### first hidden layer matched to inputs - and use a second as the data has arbitrary
#### thresholds
#### (number of features is a good number of neurons per layer - deeper gives more sophistication)
    25   : 0.7887734639019761 - 0.7809109537904840 -         06          06

   100   : 0.7947890140674736 - 0.7922373078596026 -       0:46        0:33

   500   : 0.8675915246886190 -                            4:44

  2500   : 0.7923683126864547 - 0.8622292552558385         0:39 ?      8:34
 
#### hidden_layer_sizes=(20,10,4),
#### early_stopping=True,  

    25   : 0.8329299620790784 - 0.7815451498511101         0:07        0:07
 
   100   : 

#### try verbose=True on a small example


#### hidden_layer_sizes=(20,10,5),
#### learning_rate_init=0.001,
#### early_stopping=True,

    25   : 0.19057347109521827  0:07

   100   : 0.78176263624963760  0:33  

   500   : 0.85071602031931740  5:39

  2500   : 0.85154263342489110  2:29 276


#### hidden_layer_sizes=(20,20),

  2500

#### add some more loops 
#### and complete investigation of data = ref above

for iters in [25, 100, 500] :
  for hidden in [(10,10), (15,10), (20, 10), (20,20), (20,10,5), (20,20,10)] :
    print ("\n    ",hidden,iters)
    print("      Fitting model and checking score on Known data" , iters, "iterations")
    Known = train_norm.copy()
    y = train_df["Income"].copy()


    #mod = linear_model.BayesianRidge(copy_X=False, n_iter=1000)
    mod = nnet.MLPRegressor(
        hidden_layer_sizes= hidden,
        tol=0.00001, n_iter_no_change=25,
        warm_start=False, early_stopping=True,
        learning_rate="adaptive", learning_rate_init=0.001,
        max_iter=iters)
    dtstart = dt.now()
    print("      model created ", dt.now())
    mod.fit(Known, y)
    print("      model   fit   ", dt.now())

    print("      ", mod.score(Known, y))

    print ("      ", dt.now() - dtstart)
    print ("      ", mod.n_iter_)

    # print("      ", dt.now())
  
  #### end for
  print('end   ', hidden, dt.now())
#### end for
print('end', iters, dt.now())

Summary : no models converging at 500 iters - - set max to 1000 - want some to converge
 more complex models are producing better early results - buy evening out later - but still somewhat better
 the third layer is definitely helping - try four but overfitting is a prob here
 nodes per layer is not doing much - anything greater than feature count doesn't seen useful


     (10, 10) 25
      Fitting model and checking score on Known data 25 iterations
      model created  2019-10-10 09:33:31.360755
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:33:37.181152
       0.04900702143306124
       0:00:05.857442
       25

     (15, 10) 25
      Fitting model and checking score on Known data 25 iterations
      model created  2019-10-10 09:33:37.224805
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:33:43.423821
       0.055618036857014164
       0:00:06.236897
       25

     (20, 10) 25
      Fitting model and checking score on Known data 25 iterations
      model created  2019-10-10 09:33:43.468912
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:33:49.711396
       0.08391504732899813
       0:00:06.285070
       25

     (20, 20) 25
      Fitting model and checking score on Known data 25 iterations
      model created  2019-10-10 09:33:49.761121
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:33:56.333045
       0.0845487845042745
       0:00:06.621983
       25

     (20, 10, 5) 25
      Fitting model and checking score on Known data 25 iterations
      model created  2019-10-10 09:33:56.390643
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:34:03.908871
       0.5890762607476452
       0:00:07.566957
       25

     (20, 20, 10) 25
      Fitting model and checking score on Known data 25 iterations
      model created  2019-10-10 09:34:03.966853
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:34:12.095002
       0.7680736717882629
       0:00:08.189435
       25
end    (20, 20, 10) 2019-10-10 09:34:12.156425

     (10, 10) 100
      Fitting model and checking score on Known data 100 iterations
      model created  2019-10-10 09:34:12.164679
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:34:40.489466
       0.7631855019633931
       0:00:28.376282
       100

     (15, 10) 100
      Fitting model and checking score on Known data 100 iterations
      model created  2019-10-10 09:34:40.550196
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:35:12.858821
       0.7381185626688946
       0:00:32.376740
       100

     (20, 10) 100
      Fitting model and checking score on Known data 100 iterations
      model created  2019-10-10 09:35:12.935919
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:35:40.034723
       0.7636492674828678
       0:00:27.148139
       100

     (20, 20) 100
      Fitting model and checking score on Known data 100 iterations
      model created  2019-10-10 09:35:40.093492
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:36:24.381839
       0.7786494832713816
       0:00:44.390218
       100

     (20, 10, 5) 100
      Fitting model and checking score on Known data 100 iterations
      model created  2019-10-10 09:36:24.493112
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:37:06.415136
       0.7895194173603781
       0:00:42.014290
       100

     (20, 20, 10) 100
      Fitting model and checking score on Known data 100 iterations
      model created  2019-10-10 09:37:06.516429
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:37:58.232064
       0.7920320608715972
       0:00:51.830771
       100
end    (20, 20, 10) 2019-10-10 09:37:58.347585

     (10, 10) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 09:37:58.356871
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:41:55.503532
       0.7980891796367898
       0:03:57.245916
       500

     (15, 10) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 09:41:55.611581
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:45:27.954997
       0.8315259032349603
       0:03:32.439927
       500

     (20, 10) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 09:45:28.060818
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:49:42.244395
       0.7926614899392772
       0:04:14.301349
       500

     (20, 20) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 09:49:42.370786
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 09:54:40.780235
       0.7934448999872588
       0:04:58.561461
       500

     (20, 10, 5) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 09:54:40.940533
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 10:00:09.746135
       0.8455746791204602
       0:05:28.962986
       500

     (20, 20, 10) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 10:00:09.911843
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 10:07:40.174202
       0.8564149216340179
       0:07:30.512693
       500
end    (20, 20, 10) 2019-10-10 10:07:40.424942
end 500 2019-10-10 10:07:40.425557

#### add some more loops 
#### and complete investigation of data = ref above

for iters in [ 200, 500, 1000] :
  for hidden in [(5,), (10,), (20,), (30,),
                 (10,5), (10,10), (15,10), (20, 10), (20,20),
                 (15,10,5), (20,10,5), (20,20,10), (20,20,20), (30,20,20),
                 (30,20,10,5), (40,30,20,10)] :
    print ("\n    ",hidden,iters)
    print("      Fitting model and checking score on Known data" , iters, "iterations")
    Known = train_norm.copy()
    y = train_df["Income"].copy()


    #mod = linear_model.BayesianRidge(copy_X=False, n_iter=1000)
    mod = nnet.MLPRegressor(
        hidden_layer_sizes= hidden,
        tol=0.00001, n_iter_no_change=25,
        warm_start=False, early_stopping=True,
        learning_rate="adaptive", learning_rate_init=0.001,
        max_iter=iters)
    dtstart = dt.now()
    print("      model created ", dt.now())
    mod.fit(Known, y)
    print("      model   fit   ", dt.now())

    print("      ", mod.score(Known, y))

    print ("      ", dt.now() - dtstart)
    print ("      ", mod.n_iter_)

    # print("      ", dt.now())
  
  #### end for
  print('end   ', hidden, dt.now())
#### end for
print('end', iters, dt.now())


     (5,) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 10:47:53.279856
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 10:48:34.999418
       0.0301556793972817
       0:00:41.767894
       200

     (10,) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 10:48:35.057482
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 10:49:34.629549
       0.04285950838693875
       0:00:59.673996
       200

     (20,) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 10:49:34.739504
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 10:51:02.904169
       0.07462261255991953
       0:01:28.324141
       200

     (30,) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 10:51:03.072470
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 10:52:40.152883
       0.09203066164729579
       0:01:37.260335
       200

     (10, 5) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 10:52:40.340696
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 10:53:45.957755
       0.7832534740463617
       0:01:05.684780
       200

     (10, 10) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 10:53:46.033686
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 10:54:40.955478
       0.7850835855529952
       0:00:54.985810
       200

     (15, 10) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 10:54:41.026800
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 10:56:16.265144
       0.7673073991327266
       0:01:35.330512
       200

     (20, 10) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 10:56:16.365361
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 10:57:33.593557
       0.7889710988838171
       0:01:17.315888
       200

     (20, 20) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 10:57:33.688786
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 10:59:34.324738
       0.7913538372356332
       0:02:00.801470
       200

     (15, 10, 5) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 10:59:34.498081
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:01:34.864933
       0.7976181864823924
       0:02:00.530229
       200

     (20, 10, 5) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 11:01:35.035162
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:03:22.277670
       0.7737864276447246
       0:01:47.415397
       200

     (20, 20, 10) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 11:03:22.459616
      model   fit    2019-10-10 11:03:53.431633
       0.7733266447391471
       0:00:31.034126
       51

     (20, 20, 20) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 11:03:53.501830
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:07:11.690534
       0.7937762169947458
       0:03:18.476095
       200

     (30, 20, 20) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 11:07:11.985765
      model   fit    2019-10-10 11:07:40.799254
       0.7750146059353649
       0:00:28.897938
       46

     (30, 20, 10, 5) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 11:07:40.892631
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:09:58.090239
       0.8568734197234469
       0:02:17.381845
       200

     (40, 30, 20, 10) 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 11:09:58.282941
      model   fit    2019-10-10 11:14:02.810882
       0.8626491700410276
       0:04:04.942987
       142
end    (40, 30, 20, 10) 2019-10-10 11:14:03.226315

     (5,) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 11:14:03.234809
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:16:04.651846
       0.09903122752584081
       0:02:01.463704
       500

     (10,) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 11:16:04.706154
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:19:24.725476
       0.13117170821488644
       0:03:20.122126
       500

     (20,) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 11:19:24.836969
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:23:33.214167
       0.23202836565720794
       0:04:08.512761
       500

     (30,) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 11:23:33.357707
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:30:54.653542
       0.2704900449221047
       0:07:21.552933
       500

     (10, 5) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 11:30:54.918668
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:33:51.424327
       0.7720991503694135
       0:02:56.588295
       500

     (10, 10) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 11:33:51.514666
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:36:47.930751
       0.7772539269656454
       0:02:56.492939
       500

     (15, 10) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 11:36:48.015299
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:40:40.848851
       0.792650835667579
       0:03:52.931769
       500

     (20, 10) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 11:40:40.955014
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:45:44.520795
       0.7931742089692261
       0:05:03.697908
       500

     (20, 20) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 11:45:44.660703
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:51:33.721849
       0.7932036862072729
       0:05:49.242339
       500

     (15, 10, 5) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 11:51:33.911474
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 11:55:59.999190
       0.8292438695614517
       0:04:26.202548
       500

     (20, 10, 5) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 11:56:00.122033
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 12:00:43.992710
       0.8584364840583849
       0:04:43.980424
       500

     (20, 20, 10) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 12:00:44.110394
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 12:06:53.798825
       0.8625297588193361
       0:06:09.861295
       500

     (20, 20, 20) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 12:06:53.980181
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 12:14:39.013617
       0.8576937540307676
       0:07:45.299008
       500

     (30, 20, 20) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 12:14:39.287546
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 12:24:53.280863
       0.8493605467070405
       0:10:14.335360
       500

     (30, 20, 10, 5) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 12:24:53.631465
      model   fit    2019-10-10 12:32:02.203261
       0.8630160098707162
       0:07:08.868963
       327

     (40, 30, 20, 10) 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 12:32:02.509804
      model   fit    2019-10-10 12:36:29.186057
       0.861921476374926
       0:04:27.114403
       166
end    (40, 30, 20, 10) 2019-10-10 12:36:29.624619

     (5,) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 12:36:29.633280
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 12:40:49.709176
       0.25959580135316507
       0:04:20.125678
       1000

     (10,) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 12:40:49.767559
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 12:45:19.223582
       0.49181449517528864
       0:04:29.507899
       1000

     (20,) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 12:45:19.283679
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 12:52:41.766734
       0.6848908729049296
       0:07:22.589052
       1000

     (30,) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 12:52:41.881513
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 13:03:15.167647
       0.7259753694756534
       0:10:33.452365
       1000

     (10, 5) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 13:03:15.341926
      model   fit    2019-10-10 13:03:21.657213
       -0.531387441515947
       0:00:06.346316
       27

     (10, 10) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 13:03:21.695385
      model   fit    2019-10-10 13:07:04.630560
       0.7910691164220829
       0:03:43.050278
       438

     (15, 10) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 13:07:04.752840
      model   fit    2019-10-10 13:08:01.169633
       0.7814409610428632
       0:00:56.496408
       160

     (20, 10) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 13:08:01.255946
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
      model   fit    2019-10-10 13:17:47.886507
       0.7960003670605276
       0:09:46.763366
       1000

     (20, 20) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 13:17:48.026805
      model   fit    2019-10-10 13:25:21.638235
       0.7937293152719301
       0:07:33.798122
       593

     (15, 10, 5) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 13:25:21.833161
      model   fit    2019-10-10 13:29:24.716626
       0.8568137796510888
       0:04:02.978032
       504

     (20, 10, 5) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 13:29:24.819418
      model   fit    2019-10-10 13:30:08.998374
       0.7839810296899453
       0:00:44.255734
       104

     (20, 20, 10) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 13:30:09.082897
      model   fit    2019-10-10 13:37:56.412566
       0.8602100046874072
       0:07:47.555383
       481

     (20, 20, 20) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 13:37:56.646549
      model   fit    2019-10-10 13:44:58.933620
       0.861849639610157
       0:07:02.564611
       441

     (30, 20, 20) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 13:44:59.218951
      model   fit    2019-10-10 14:01:50.655469
       0.8629787727866067
       0:16:51.724465
       834

     (30, 20, 10, 5) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 14:01:50.951668
      model   fit    2019-10-10 14:02:03.020741
       -0.5313979009896326
       0:00:12.164888
       27

     (40, 30, 20, 10) 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 14:02:03.124643
      model   fit    2019-10-10 14:10:38.939692
       0.8647387360072559
       0:08:36.349094
       214
end    (40, 30, 20, 10) 2019-10-10 14:10:39.474205
end 1000 2019-10-10 14:10:39.474884

====================================================================

#### add some more loops 
#### and complete investigation of data = ref above


#### for hidden in [(5,), (10,), (20,), (30,), (10,5), (10,10), (15,10), (20, 10), (20,20), (15,10,5), (20,10,5), (20,20,10), (20,20,20), (30,20,20), (30,20,10,5), (40,30,20,10)] :
for dpth in [2,3,4,5,6,7,8,9,10] :
  for iters in [200, 500, 1000] :
    print ("\n    ",dpth,iters)
    # print ("\n    ",hidden,iters)
    print("      Fitting model and checking score on Known data" , iters, "iterations")
    Known = train_norm.copy()
    y = train_df["Income"].copy()


    #mod = linear_model.BayesianRidge(copy_X=False, tol=0.00001, n_iter=1000)
    # 0.7349053641228902
        
    # mod = nnet.MLPRegressor(
    #     hidden_layer_sizes= hidden,
    #     tol=0.00001, n_iter_no_change=25,
    #     warm_start=False, early_stopping=True,
    #     learning_rate="adaptive", learning_rate_init=0.001,
    #     max_iter=iters)
        
    # mod = ensemble.AdaBoostRegressor( n_estimators=100 )
    # doesn't work   
    
    # mod = svm.LinearSVR( tol=1e-5, max_iter =10000)
    # doesn't work
    
    # mod = ensemble.RandomForestRegressor(max_depth=7, n_estimators=1000)
    # 0.8288730954198963 in 6:06 - (max_depth=7, n_estimators=1000)
    mod = ensemble.RandomForestRegressor(max_depth=dpth, n_estimators=(iters*2))
    
    
    dtstart = dt.now()
    print("      model created ", dt.now())
    mod.fit(Known, y)
    print("      model   fit   ", dt.now())

    print("      ", mod.score(Known, y))

    print ("      ", dt.now() - dtstart)
    # print ("      ", mod.n_iter_)

    # print("      ", dt.now())
  
  #### end for
  print('end   ', hidden, dt.now())
#### end for
print('end', iters, dt.now())


     2 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 20:21:14.378696
      model   fit    2019-10-10 20:21:53.271866
       0.5736229175179561
       0:00:39.625358

     2 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 20:21:54.010832
      model   fit    2019-10-10 20:23:31.236617
       0.5741663052074726
       0:01:39.011410

     2 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 20:23:33.030093
      model   fit    2019-10-10 20:26:47.926798
       0.573818832838402
       0:03:18.454079
end    (5,) 2019-10-10 20:26:51.484233

     3 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 20:26:51.492473
      model   fit    2019-10-10 20:27:49.851387
       0.6468803160380183
       0:00:59.477994

     3 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 20:27:50.977921
      model   fit    2019-10-10 20:30:18.564119
       0.647037094860585
       0:02:30.352967

     3 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 20:30:21.338211
      model   fit    2019-10-10 20:35:10.315396
       0.646929737395626
       0:04:54.480224
end    (5,) 2019-10-10 20:35:15.818480

     4 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 20:35:15.826252
      model   fit    2019-10-10 20:36:33.224299
       0.71350731028145
       0:01:18.926923

     4 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 20:36:34.760355
      model   fit    2019-10-10 20:39:48.039133
       0.7134267631787063
       0:03:17.100182

     4 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 20:39:51.868232
      model   fit    2019-10-10 20:46:20.172737
       0.7138412086345978
       0:06:35.992590
end    (5,) 2019-10-10 20:46:27.861367

     5 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 20:46:27.870368
      model   fit    2019-10-10 20:48:04.621877
       0.7638178338532969
       0:01:38.736253

     5 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 20:48:06.613606
      model   fit    2019-10-10 20:52:06.734462
       0.7646974230780095
       0:04:04.998221

     5 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 20:52:11.620979
      model   fit    2019-10-10 21:00:25.314945
       0.764601623113888
       0:08:23.646865
end    (5,) 2019-10-10 21:00:35.267901

     6 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 21:00:35.277276
      model   fit    2019-10-10 21:02:35.534353
       0.7997715178561466
       0:02:02.763652

     6 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 21:02:38.049580
      model   fit    2019-10-10 21:07:31.927603
       0.8000867207667924
       0:05:00.144226

     6 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 21:07:38.203885
      model   fit    2019-10-10 21:17:31.549466
       0.8000346637288746
       0:10:05.545628
end    (5,) 2019-10-10 21:17:43.749597

     7 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 21:17:43.761069
      model   fit    2019-10-10 21:20:07.109243
       0.8290575989685841
       0:02:26.446109

     7 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 21:20:10.217641
      model   fit    2019-10-10 21:26:08.829432
       0.828835231317038
       0:06:06.163957

     7 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 21:26:16.391610
      model   fit    2019-10-10 21:38:09.373842
       0.8289241629173437
       0:12:08.234684
end    (5,) 2019-10-10 21:38:24.626338

     8 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 21:38:24.642633
      model   fit    2019-10-10 21:41:07.478943
       0.8520613508354745
       0:02:46.457555

     8 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 21:41:11.111212
      model   fit    2019-10-10 21:47:58.171818
       0.8523351397135295
       0:06:56.118656

     8 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 21:48:07.243463
      model   fit    2019-10-10 22:02:35.772508
       0.8521770569895661
       0:14:47.289448
end    (5,) 2019-10-10 22:02:54.532980

     9 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 22:02:54.562003
      model   fit    2019-10-10 22:06:05.381423
       0.8750031721060594
       0:03:15.244867

     9 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 22:06:09.823157
      model   fit    2019-10-10 22:14:09.328025
       0.8748725402808654
       0:08:10.449822

     9 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 22:14:20.296168
      model   fit    2019-10-10 22:30:32.273443
       0.8747737504863684
       0:16:33.890412
end    (5,) 2019-10-10 22:30:54.186726

     10 200
      Fitting model and checking score on Known data 200 iterations
      model created  2019-10-10 22:30:54.222973
      model   fit    2019-10-10 22:34:21.119760
       0.8965588330934611
       0:03:32.017769

     10 500
      Fitting model and checking score on Known data 500 iterations
      model created  2019-10-10 22:34:26.259523
      model   fit    2019-10-10 22:43:03.942714
       0.8967560340536448
       0:08:50.348551

     10 1000
      Fitting model and checking score on Known data 1000 iterations
      model created  2019-10-10 22:43:16.639438
      model   fit    2019-10-10 22:59:36.513166
       0.8968737557325714
       0:16:44.478299
end    (5,) 2019-10-10 23:00:01.117808
end 1000 2019-10-10 23:00:01.118255

======================================================